# Combine All Analysis Results

This notebook consolidates all analysis results into a single comprehensive dataframe:
1. Late fusion model results (PRE/POST)
2. Ambiguity group analysis (balanced/unbalanced)
3. Reaction time group analysis (balanced/unbalanced)
4. Statistical testing results
5. Feature importance (SHAP) results
6. Rolling window analysis

In [1]:
import pandas as pd
import pickle
from pathlib import Path
import numpy as np

# Base paths
BASE_PATH = Path('../../data/results')

print("Libraries loaded successfully!")

Libraries loaded successfully!


## Define Loading Functions

In [2]:
def load_fusion_results(time_period):
    """Load late fusion model results for PRE or POST."""
    path = BASE_PATH / f'fusion_model_results_{time_period}'
    results = []

    # Method comparison
    comp_file = path / f'late_fusion_model_{time_period}_method_comparison.csv'
    if comp_file.exists():
        df = pd.read_csv(comp_file)
        df['analysis_type'] = 'late_fusion'
        df['time_period'] = time_period
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = comp_file.name
        results.append(df)

    # Subject accuracies
    subj_file = path / f'late_fusion_model_{time_period}_subject_accuracies.csv'
    if subj_file.exists():
        df = pd.read_csv(subj_file)
        df['analysis_type'] = 'late_fusion_subject'
        df['time_period'] = time_period
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = subj_file.name
        results.append(df)

    # Modality weights
    weight_file = path / f'late_fusion_model_{time_period}_modality_weights.csv'
    if weight_file.exists():
        df = pd.read_csv(weight_file)
        df['analysis_type'] = 'modality_weights'
        df['time_period'] = time_period
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = weight_file.name
        results.append(df)

    # Visit-based analysis
    visit_file = path / f'late_fusion_by_visit_{time_period}_comparison.csv'
    if visit_file.exists():
        df = pd.read_csv(visit_file)
        df['analysis_type'] = 'late_fusion_by_visit'
        df['time_period'] = time_period
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = visit_file.name
        results.append(df)

    return results

def load_ambiguity_results(time_period, balanced=False):
    """Load ambiguity group analysis results."""
    path = BASE_PATH / f'fusion_model_results_{time_period}'
    balance_str = '_balanced' if balanced else ''
    results = []

    for group in ['Low', 'Medium', 'High']:
        # Comparison file
        comp_file = path / f'ambiguity_group_late_fusion{balance_str}_{time_period}_{group}_comparison.csv'
        if comp_file.exists():
            df = pd.read_csv(comp_file)
            df['analysis_type'] = 'ambiguity_group'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = comp_file.name
            results.append(df)

        # Subject accuracies
        subj_file = path / f'ambiguity_group_late_fusion{balance_str}_{time_period}_{group}_subject_accuracies.csv'
        if subj_file.exists():
            df = pd.read_csv(subj_file)
            df['analysis_type'] = 'ambiguity_group_subject'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = subj_file.name
            results.append(df)

        # Weights
        weight_file = path / f'ambiguity_group_late_fusion{balance_str}_{time_period}_{group}_weights.csv'
        if weight_file.exists():
            df = pd.read_csv(weight_file)
            df['analysis_type'] = 'ambiguity_group_weights'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = weight_file.name
            results.append(df)

    return results

def load_rt_results(time_period, balanced=False):
    """Load reaction time group analysis results."""
    path = BASE_PATH / f'fusion_model_results_{time_period}'
    balance_str = '_balanced' if balanced else ''
    results = []

    for group in ['Fast', 'Slow']:
        # Comparison file
        comp_file = path / f'reaction_time_group_late_fusion{balance_str}_{time_period}_{group}_comparison.csv'
        if comp_file.exists():
            df = pd.read_csv(comp_file)
            df['analysis_type'] = 'reaction_time_group'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = comp_file.name
            results.append(df)

        # Subject accuracies
        subj_file = path / f'reaction_time_group_late_fusion{balance_str}_{time_period}_{group}_subject_accuracies.csv'
        if subj_file.exists():
            df = pd.read_csv(subj_file)
            df['analysis_type'] = 'reaction_time_group_subject'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = subj_file.name
            results.append(df)

        # Weights
        weight_file = path / f'reaction_time_group_late_fusion{balance_str}_{time_period}_{group}_weights.csv'
        if weight_file.exists():
            df = pd.read_csv(weight_file)
            df['analysis_type'] = 'reaction_time_group_weights'
            df['time_period'] = time_period
            df['class_weighting'] = 'balanced' if balanced else 'unbalanced'
            df['subgroup'] = group
            df['file_source'] = weight_file.name
            results.append(df)

    return results

def load_statistical_results(time_period):
    """Load statistical testing results."""
    path = BASE_PATH / f'analysis_outputs_{time_period}'
    results = []

    # McNemar's test summary
    mcnemar_file = path / f'mcnemar_test_summary_{time_period}.csv'
    if mcnemar_file.exists():
        df = pd.read_csv(mcnemar_file)
        df['analysis_type'] = 'mcnemar_test'
        df['time_period'] = time_period
        df['class_weighting'] = 'balanced'
        df['subgroup'] = 'none'
        df['file_source'] = mcnemar_file.name
        results.append(df)

    # Comprehensive statistical summary
    stat_file = path / f'statistical_testing_summary_{time_period}.csv'
    if stat_file.exists():
        df = pd.read_csv(stat_file)
        df['analysis_type'] = 'statistical_summary'
        df['time_period'] = time_period
        df['class_weighting'] = 'balanced'
        df['subgroup'] = 'none'
        df['file_source'] = stat_file.name
        results.append(df)

    return results

def load_shap_results(time_period):
    """Load SHAP feature importance results."""
    path = BASE_PATH / f'analysis_outputs_{time_period}'
    results = []

    modalities = ['all', 'behavior', 'gaze', 'physiology', 'combined', 'phys_gaze']

    for modality in modalities:
        shap_file = path / f'shap_importance_{modality}_{time_period}.csv'
        if shap_file.exists():
            df = pd.read_csv(shap_file)
            df['analysis_type'] = f'shap_{modality}'
            df['time_period'] = time_period
            df['class_weighting'] = 'na'
            df['subgroup'] = 'none'
            df['file_source'] = shap_file.name
            results.append(df)

    return results

def load_rolling_window_results():
    """Load rolling window post-decision analysis results."""
    path = BASE_PATH / 'fusion_model_results_POST'
    results = []

    # Summary
    summary_file = path / 'rolling_window_post_decision_summary.csv'
    if summary_file.exists():
        df = pd.read_csv(summary_file)
        df['analysis_type'] = 'rolling_window_summary'
        df['time_period'] = 'POST_rolling'
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = summary_file.name
        results.append(df)

    # Detailed analysis
    detail_file = path / 'rolling_window_post_decision_analysis.csv'
    if detail_file.exists():
        df = pd.read_csv(detail_file)
        df['analysis_type'] = 'rolling_window_detailed'
        df['time_period'] = 'POST_rolling'
        df['class_weighting'] = 'unbalanced'
        df['subgroup'] = 'none'
        df['file_source'] = detail_file.name
        results.append(df)

    return results

print("Functions defined successfully!")

Functions defined successfully!


## Load All Results

In [3]:
print("="*70)
print("COMBINING ALL ANALYSIS RESULTS")
print("="*70)

all_results = []

# 1. Late fusion results
print("\n1. Loading late fusion results...")
for time_period in ['PRE', 'POST']:
    results = load_fusion_results(time_period)
    all_results.extend(results)
    print(f"   ✓ Loaded {len(results)} files for {time_period}")

# 2. Ambiguity group results
print("\n2. Loading ambiguity group results...")
for time_period in ['PRE', 'POST']:
    for balanced in [False, True]:
        results = load_ambiguity_results(time_period, balanced)
        all_results.extend(results)
        balance_str = 'balanced' if balanced else 'unbalanced'
        print(f"   ✓ Loaded {len(results)} files for {time_period} ({balance_str})")

# 3. Reaction time group results
print("\n3. Loading reaction time group results...")
for time_period in ['PRE', 'POST']:
    for balanced in [False, True]:
        results = load_rt_results(time_period, balanced)
        all_results.extend(results)
        balance_str = 'balanced' if balanced else 'unbalanced'
        print(f"   ✓ Loaded {len(results)} files for {time_period} ({balance_str})")

# 4. Statistical testing results
print("\n4. Loading statistical testing results...")
for time_period in ['PRE', 'POST']:
    results = load_statistical_results(time_period)
    all_results.extend(results)
    print(f"   ✓ Loaded {len(results)} files for {time_period}")

# 5. SHAP feature importance
print("\n5. Loading SHAP feature importance results...")
for time_period in ['PRE', 'POST']:
    results = load_shap_results(time_period)
    all_results.extend(results)
    print(f"   ✓ Loaded {len(results)} files for {time_period}")

# 6. Rolling window results
print("\n6. Loading rolling window results...")
results = load_rolling_window_results()
all_results.extend(results)
print(f"   ✓ Loaded {len(results)} files")

print(f"\n{'='*70}")
print(f"Total dataframes loaded: {len(all_results)}")

COMBINING ALL ANALYSIS RESULTS

1. Loading late fusion results...
   ✓ Loaded 4 files for PRE
   ✓ Loaded 4 files for POST

2. Loading ambiguity group results...
   ✓ Loaded 6 files for PRE (unbalanced)
   ✓ Loaded 6 files for PRE (balanced)
   ✓ Loaded 6 files for POST (unbalanced)
   ✓ Loaded 6 files for POST (balanced)

3. Loading reaction time group results...
   ✓ Loaded 4 files for PRE (unbalanced)
   ✓ Loaded 4 files for PRE (balanced)
   ✓ Loaded 4 files for POST (unbalanced)
   ✓ Loaded 4 files for POST (balanced)

4. Loading statistical testing results...
   ✓ Loaded 0 files for PRE
   ✓ Loaded 0 files for POST

5. Loading SHAP feature importance results...
   ✓ Loaded 6 files for PRE
   ✓ Loaded 0 files for POST

6. Loading rolling window results...
   ✓ Loaded 0 files

Total dataframes loaded: 54


## Combine and Save Results

In [4]:
# Concatenate all dataframes
combined_df = pd.concat(all_results, ignore_index=True, sort=False)

print(f"✓ Combined dataframe shape: {combined_df.shape}")
print(f"\nAnalysis types included:")
for analysis_type in sorted(combined_df['analysis_type'].unique()):
    count = (combined_df['analysis_type'] == analysis_type).sum()
    print(f"  - {analysis_type}: {count} rows")

# Save combined results
output_file = BASE_PATH / 'combined_all_results.csv'
combined_df.to_csv(output_file, index=False)
print(f"\n{'='*70}")
print(f"✓ SAVED: {output_file}")
print(f"{'='*70}")

✓ Combined dataframe shape: (2364, 24)

Analysis types included:
  - ambiguity_group_subject: 1164 rows
  - ambiguity_group_weights: 36 rows
  - late_fusion: 12 rows
  - late_fusion_by_visit: 6 rows
  - late_fusion_subject: 194 rows
  - modality_weights: 6 rows
  - reaction_time_group_subject: 776 rows
  - reaction_time_group_weights: 24 rows
  - shap_all: 40 rows
  - shap_behavior: 7 rows
  - shap_combined: 33 rows
  - shap_gaze: 20 rows
  - shap_phys_gaze: 33 rows
  - shap_physiology: 13 rows

✓ SAVED: ../../data/results/combined_all_results.csv


## Summary Statistics

In [5]:
# Print summary statistics
print("\nSUMMARY BY DIMENSIONS:")
print(f"\n1. Analysis Types: {combined_df['analysis_type'].nunique()}")
print(f"   {sorted(combined_df['analysis_type'].unique())}")

print(f"\n2. Time Periods: {combined_df['time_period'].nunique()}")
print(f"   {sorted(combined_df['time_period'].unique())}")

print(f"\n3. Class Weighting: {combined_df['class_weighting'].nunique()}")
print(f"   {sorted(combined_df['class_weighting'].unique())}")

print(f"\n4. Subgroups: {combined_df['subgroup'].nunique()}")
print(f"   {sorted(combined_df['subgroup'].unique())}")


SUMMARY BY DIMENSIONS:

1. Analysis Types: 14
   ['ambiguity_group_subject', 'ambiguity_group_weights', 'late_fusion', 'late_fusion_by_visit', 'late_fusion_subject', 'modality_weights', 'reaction_time_group_subject', 'reaction_time_group_weights', 'shap_all', 'shap_behavior', 'shap_combined', 'shap_gaze', 'shap_phys_gaze', 'shap_physiology']

2. Time Periods: 2
   ['POST', 'PRE']

3. Class Weighting: 3
   ['balanced', 'na', 'unbalanced']

4. Subgroups: 6
   ['Fast', 'High', 'Low', 'Medium', 'Slow', 'none']


## Create Pivot Summary

In [6]:
print(f"\n{'='*70}")
print("CREATING PIVOT SUMMARY...")
print(f"{'='*70}")

# Create summary by analysis type
summary_data = []

for analysis_type in sorted(combined_df['analysis_type'].unique()):
    subset = combined_df[combined_df['analysis_type'] == analysis_type]
    
    summary_data.append({
        'analysis_type': analysis_type,
        'n_rows': len(subset),
        'time_periods': ', '.join(sorted(subset['time_period'].unique())),
        'class_weighting': ', '.join(sorted(subset['class_weighting'].unique())),
        'subgroups': ', '.join(sorted(subset['subgroup'].unique())),
        'n_files': subset['file_source'].nunique()
    })

summary_df = pd.DataFrame(summary_data)
summary_file = BASE_PATH / 'combined_results_summary.csv'
summary_df.to_csv(summary_file, index=False)

print(f"\n✓ SAVED SUMMARY: {summary_file}")
print("\n" + summary_df.to_string(index=False))


CREATING PIVOT SUMMARY...

✓ SAVED SUMMARY: ../../data/results/combined_results_summary.csv

              analysis_type  n_rows time_periods      class_weighting         subgroups  n_files
    ambiguity_group_subject    1164    POST, PRE balanced, unbalanced High, Low, Medium       12
    ambiguity_group_weights      36    POST, PRE balanced, unbalanced High, Low, Medium       12
                late_fusion      12    POST, PRE           unbalanced              none        2
       late_fusion_by_visit       6    POST, PRE           unbalanced              none        2
        late_fusion_subject     194    POST, PRE           unbalanced              none        2
           modality_weights       6    POST, PRE           unbalanced              none        2
reaction_time_group_subject     776    POST, PRE balanced, unbalanced        Fast, Slow        8
reaction_time_group_weights      24    POST, PRE balanced, unbalanced        Fast, Slow        8
                   shap_all      

## Display Sample of Combined Data

In [7]:
# Show first few rows
print("\nFirst 10 rows of combined data:")
combined_df.head(10)


First 10 rows of combined data:


,Method,Accuracy,F1-Score,analysis_type,time_period,class_weighting,subgroup,file_source,subject_id,accuracy,...,Visit,N_subjects,N_trials,F1_Score,Mean_Subject_Acc,Std_Subject_Acc,f1_score,Weight,feature,mean_abs_shap
0,Physiology Only,0.519080,0.504044,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Behavior Only,0.647486,0.660967,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gaze Only,0.502747,0.504944,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Average Fusion,0.638032,0.639036,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Weighted Fusion,0.693054,0.679360,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Stacking,0.635366,0.644421,late_fusion,PRE,unbalanced,none,late_fusion_model_PRE_method_comparison.csv,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,late_fusion_subject,PRE,unbalanced,none,late_fusion_model_PRE_subject_accuracies.csv,0806_1000_539136F,0.635593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,late_fusion_subject,PRE,unbalanced,none,late_fusion_model_PRE_subject_accuracies.csv,0806_1000_U9TEJGM,0.725191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,late_fusion_subject,PRE,unbalanced,none,late_fusion_model_PRE_subject_accuracies.csv,0811_1000_4LI8GO7,0.595041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,late_fusion_subject,PRE,unbalanced,none,late_fusion_model_PRE_subject_accuracies.csv,0811_1000_539136F,0.738462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Show column names
print("\nColumns in combined dataset:")
print(combined_df.columns.tolist())


Columns in combined dataset:
['Method', 'Accuracy', 'F1-Score', 'analysis_type', 'time_period', 'class_weighting', 'subgroup', 'file_source', 'subject_id', 'accuracy', 'Modality', 'Average', 'Weighted', 'Stacking', 'Visit', 'N_subjects', 'N_trials', 'F1_Score', 'Mean_Subject_Acc', 'Std_Subject_Acc', 'f1_score', 'Weight', 'feature', 'mean_abs_shap']
